In [ ]:
# Basic python import
import os
import sys
import yaml
import matplotlib.pyplot as plt
import pandas as pd

# Pytorch
import torch
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn

# Our custom module
sys.path.append('../')
import data_processing
import models
import evaluation 
import submission

# Config

Ma proposition : 
Avoir un Notebook template associé à des paramètres par défaut. 
Ensuite pour chaque expérience on le duplique, et on change les valeurs des paramètres que l'on souhaite.

In [ ]:
with open('../configs/default_params.yaml') as default_params_file:
      default_params = yaml.safe_load(default_params_file)
params = default_params
params

data_dir = os.path.join('..','data')

torch.manual_seed(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')

In [ ]:
# Changer de paramètres
params["train"]["batch_size"] = 1
params["train"]

# Export masks as .tiff files

If it's the first time you run this notebook, you should uncomment the following cell and run it. It will read the masks from the .csv file and output them as .tiff files in a "train_masks" folder

In [ ]:
from data_processing.preproc import create_masks_as_tiff, preprocess_images_and_masks

# create_masks_as_tiff(data_dir)


# Create Dataset and Dataloader

In [ ]:
from data_processing.utils import get_training_datasets_and_dataloaders

train_dataset, validation_dataset, train_dataloader, validation_dataloader = get_training_datasets_and_dataloaders(batch_size=params["train"]["batch_size"] ,input_size=1024)

# Train

In [ ]:
from models.unet import UNet

MODEL = UNet(num_classes=1)
lr = 1e-3
optimizer = optim.Adam(MODEL.parameters(), lr=lr)
loss = nn.MSELoss()
optimizer

In [ ]:
from train.train import main_train, main_train_batch1

if params["train"]["batch_size"] == 1:
    loss_list = main_train_batch1(model=MODEL.to(device), loss_fn=loss, optimizer=optimizer, n_epochs=25, dataset=train_dataset, device=device)
if params["train"]["batch_size"] > 1:
    loss_list = main_train(model=MODEL.to(device), loss_fn=loss, optimizer=optimizer, n_epochs=25, dataloader=train_dataloader, device=device)

In [ ]:
MODEL.to(device)

In [ ]:
import numpy as np

def show_image_and_mask(image,mask,cmaps):
    fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(16, 32))
    hybr = image[0, :, :]/2 + mask[0, :, :]

    ax[0].imshow(image.T)
    ax[0].axis('off')
    ax[0].set_title('IMAGE')
    ax[1].imshow(hybr.T,cmap=cmaps)
    ax[1].axis('off')
    ax[1].set_title('MASK ON IMAGE')
    plt.show()

image, _, mask = train_dataset[0]

show_image_and_mask(image,mask,"gray")
#show_image_and_mask(image,MODEL(torch.unsqueeze(image, dim=0).to(device)).cpu().detach(),"gray")

# Evaluation

Running model on validation dataset

In [ ]:
val_pred = {"val_images":[], "val_masks":[], "val_organs":[], "val_y_preds":[]}
for k, (val_image, val_organ, val_mask) in enumerate(train_dataloader):
    val_pred['val_images'].append(val_image)
    val_pred['val_mask'].append(val_mask)
    val_pred['val_organ'].append(val_organ)
    pred = MODEL(val_image)
    val_pred['val_y_preds'].append(pred)

In [ ]:
train_dataset[0]

In [ ]:
IoUs, IoUsOrgans = evaluation.get_y_true_y_pred(val_pred)

# Plot results for different threshold on all training and get best threshold
best_threshold = evaluation.plot_iou_by_threshold(IoUs, 'test')

We can also look at some of the results

In [ ]:
evaluation.plot_validation_predictions(val_pred, best_threshold, 10)

# Submission

In [ ]:
test_dataset = None
submission.make_submission(MODEL, test_dataset, best_threshold)